# Bundle adjustment of $n$-cameras

### Goal

Suppose $n$ cameras have been calibrated linearly beforehand, using 2D-2D correspondences for example.  We assume we have an initial guess of $R$ and $t$ for each camera satisfying $x \sim A (R X + t)$, the 2D observation $x$, and the 3D point $X$.

* The initial guess is not necessarily done by linear methods.  It can even be done by hand.
* The 3D point $X$ can be triangulated from $x$ using the initial $R, t$.
* The $n$ cameras do not necessarily observe all the points.

Given $A, d, R, t, x, X$, this notebook optimizes $A, d, R, t, X$ so as to minimize the reprojection error.

* Input:
  * initial guess of $A, d, R, t, X$ and 2D observations $x$
* Output:
  * optimal $A, d, R, t, X$
 

## Libraries

In [1]:
%matplotlib notebook
import sys, os, cv2
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from pycalib.plot import plotCamera
from pycalib.ba import bundle_adjustment, gen_camera_param
from pycalib.calib import lookat, triangulate

## Synthetic data

In [10]:
# 3D points
# X_gt = (np.random.rand(16, 3) - 0.5)*5 # random points centered at [0, 0, 0]
X_gt = np.array(np.meshgrid(np.linspace(-1, 1, 3), np.linspace(-1, 1, 3), np.linspace(-1, 1, 3))).reshape((3, -1)).T  # 3D grid points
Np = X_gt.shape[0]
print('X_gt:', X_gt.shape)

# Camera intrinsics
K = np.array([[600, 0, 320], [0, 600, 240], [0, 0, 1]]).astype(np.float)  # VGA camera

# Camera poses: cameras are at the vertices of a hexagon
t = 2 * np.pi / 5 * np.arange(5)
v_gt = np.vstack((10*np.cos(t), 10*np.sin(t), np.zeros(t.shape))).T
Nc = v_gt.shape[0]
R_gt = []
t_gt = []
P_gt = []
rvec_gt = []
for i in range(Nc):
    t = v_gt[i,:]
    R, t = lookat(t, np.zeros(3), np.array([0, 1, 0]))
    R_gt.append(R)
    t_gt.append(t)
    P_gt.append(K @ np.hstack((R, t)))
    rvec_gt.append(cv2.Rodrigues(R)[0])
R_gt = np.array(R_gt)
t_gt = np.array(t_gt)
P_gt = np.array(P_gt)
rvec_gt = np.array(rvec_gt)
print('R_gt:', R_gt.shape)
print('t_gt:', t_gt.shape)
print('P_gt:', P_gt.shape)
print('rvec_gt:', rvec_gt.shape)

# 2D observations points
x_gt = []
for i in range(Nc):
    xt = cv2.projectPoints(X_gt.reshape((-1, 1, 3)), rvec_gt[i], t_gt[i], K, None)[0].reshape((-1, 2))
    x_gt.append(xt)
x_gt = np.array(x_gt)
print('x_gt:', x_gt.shape)

# Verify triangulation
Y = []
for i in range(Np):
    y = triangulate(x_gt[:,i,:].reshape((-1,2)), P_gt)
    #print(y)
    Y.append(y)
Y = np.array(Y).T
Y = Y[:3,:] / Y[3,:]
assert np.allclose(0, X_gt - Y.T)

# Verify z > 0 at each camera
for i in range(Nc):
    Xc = R_gt[i] @ X_gt.T + t_gt[i]
    assert np.all(Xc[2, :] > 0)

    
# Inject gaussian noise to the inital guess
R_est = R_gt.copy()
t_est = t_gt.copy()
K_est = np.array([K for c in range(Nc)])
X_est = X_gt.copy()
x_est = x_gt.copy()

for i in range(Nc):
    R_est[i] = cv2.Rodrigues( cv2.Rodrigues(R_est[i])[0] + np.random.normal(0, 0.01, (3,1)) )[0]
    t_est[i] += np.random.normal(0, 0.01, (3,1))
    K_est[i][0,0] = K_est[i][1,1] = K_est[i][0,0] + np.random.normal(0, K_est[i][0,0]/10)

X_est += np.random.normal(0, 0.01, X_est.shape)
x_est += np.random.normal(0, 0.1, x_est.shape)

X_gt: (27, 3)
R_gt: (5, 3, 3)
t_gt: (5, 3, 1)
P_gt: (5, 3, 4)
rvec_gt: (5, 3, 1)
x_gt: (5, 27, 2)


## Bundle adjustment

In [9]:
# def bundle_adjustment(camera_params, points_3d, camera_indices, point_indices, points_2d, *, verbose=2, mask=None):

# Camera parameters
camera_params = []
for i in range(Nc):
    c = gen_camera_param(R_est[i], t_est[i], K_est[i], np.zeros(5))
    camera_params.append(c)
camera_params = np.array(camera_params)

camera_indices = np.repeat(np.arange(Nc), Np)
point_indices = np.tile(np.arange(Np), Nc)

cam_opt, X_opt, ret = bundle_adjustment(camera_params, X_est, camera_indices, point_indices, x_est.reshape((-1, 2)))

print(X_gt)
print(X_opt)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.2335e+02                                    1.52e+03    
       1              2         1.2180e-01      5.23e+02       1.32e+02       9.82e+01    
       2              3         2.2982e-06      1.22e-01       6.79e+01       5.32e-02    
       3              4         3.1840e-07      1.98e-06       5.13e+01       9.94e-03    
       4              5         7.5907e-08      2.42e-07       1.29e+01       1.04e-03    
       5              6         1.9124e-08      5.68e-08       1.12e+01       1.27e-03    
       6              7         8.9441e-09      1.02e-08       2.89e+00       4.33e-04    
       7              8         7.2559e-09      1.69e-09       1.98e+00       2.71e-04    
       8              9         7.0871e-09      1.69e-10       2.28e-01       1.21e-04    
       9             10         7.0590e-09      2.81e-11       1.08e-01       5.02e-05    